In [ ]:
# 多个正态总体的均值向量和协方差阵同时检验
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import manifold
import os
import pandas as pd

In [ ]:
# 各组均值向量
def single_mean(vector):#vector为需要计算的单个正太总体（对其按行求均值）
    single_vector = np.mean(vector,axis=0)
#     print('单个均值向量%s'%single_vector)
    return single_vector  

In [ ]:
def all_mean(vector_01,vector_02,k):#k=组别数
    all_vector = np.vstack([vector_01,vector_02])
    all_vector = np.mean(all_vector,axis=0)
#     print('总均值向量%s'%all_vector)
    return all_vector

In [ ]:
# 单类别总体的离差样本阵
def single_A(vector):#输入单个类别
    single_vector = single_mean(vector)# 该组别均值向量
    constant_A = np.zeros([vector.shape[1],vector.shape[1]]) 
    for i in range(vector.shape[0]):# n_t = vector.shape[0],相当于单组别采集次数
        variable_0 = (vector[i] - single_vector).reshape(1,vector.shape[1])# 更改形状，vector.shape[1] = p = 光谱采集点数
        variable_1 = variable_0.T
        variable_2 = np.dot(variable_1,variable_0)
        constant_A = constant_A + variable_2
#     print('单个A%s'%constant_A)
    return constant_A
def all_A(vector_01,vector_02): # 全部总体的离差样本阵
    constant_all_A = single_A(vector_01)+single_A(vector_02)
#     print('总A%s'%constant_all_A)
    return constant_all_A

In [ ]:
def all_T(vector_01,vector_02, n, k):#n为全类别总共采集次数
    A = all_A(vector_01,vector_02)
    # print(A)
    all_vector = all_mean(vector_01,vector_02, k).reshape(1,vector_01.shape[1])# 整体均值向量
    # print('a%s'%all_vector)
    single_vector_01 = single_mean(vector_01).reshape(1,vector_01.shape[1])
    single_vector_02 = single_mean(vector_02).reshape(1,vector_01.shape[1])

    D1 = single_vector_01-all_vector 
    D2 = single_vector_02-all_vector
#     print('D%s'%D1)
    T = vector_01.shape[0]*np.dot(D1.T,D1)+vector_02.shape[0]*np.dot(D2.T,D2) + A# vector_01.shape[0]=n_t
#     print('D1%s'%(vector_01.shape[0]*np.dot(D1.T,D1)+vector_02.shape[0]*np.dot(D2.T,D2)))

    return T

In [ ]:
def ca_f(vector,k):# k = 组别数
    p = vector.shape[1]# p = 变量个数
    f = (p*(p+3)*(k-1))/2
    return f

In [ ]:
def ca_b(vector_01,vector_02, n, k):# k=组别数,这里默认为4,n为全类别总共采集次数
    # 1/(n_i-1)的连加
    v1 = 1/(vector_01.shape[0]-1)+1/(vector_02.shape[0]-1)
    # 1/(n-k)
    v2 = 1/(n-k)
    # (2*p^2+3p-1)/(6(p+3)(k-1)),p=vector_01.shape[1]为变量个数
    v3 = (2*math.pow(vector_01.shape[1],2)+3*vector_01.shape[1]-1)/(6*(vector_01.shape[1]+3)*(k-1))
    # 最后一项
    v4 = (vector_01.shape[1]-k+2)/((n-k)*(vector_01.shape[1]+3))
    b = (v1-v2)*(v3)-(v4)
    print('b%s'%b)
    return b

In [ ]:
def lamda_5star(vector_01,vector_02,n,k):
    # print(vector_01.shape[0])
#     print(single_A(vector_03))
#     print(np.linalg.det(single_A(vector_03)))
    s = np.linalg.det(all_T(vector_01,vector_02,n,k)/(n-k))
    s1 = np.linalg.det(single_A(vector_01)/(vector_01.shape[0]-1))
    s2 = np.linalg.det(single_A(vector_02)/(vector_02.shape[0]-1))
    print(s,s1,s2)
    T = ((n-k)/2)*np.log(s)
    A1 = ((vector_01.shape[0]-1)/2)*np.log(s1)
    A2 = ((vector_02.shape[0]-1)/2)*np.log(s2)
    lamda = (-2)*((A1+A2)-T)
    return lamda

In [ ]:
def cal_epsilon(vector_01,vector_02,n,k):
    lamda = lamda_5star(vector_01,vector_02,n,k)
    b = ca_b(vector_01,vector_02, n, k)
    epsilon = (1-b)*lamda
    return epsilon
    
    

In [ ]:
# 标准化
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

In [ ]:
os.chdir(r'E:\Major_revision\大修绘图\t-SNE')
data0 = pd.read_csv('t-SNEx1&x.csv')
data1 = pd.read_csv('t-SNEx2&x.csv')
data2 = pd.read_csv('t-SNEx3&x.csv')
data3 = pd.read_csv('t-SNEall.csv')
data0 = np.array(data0)
data0 = data0[:,0:3]
data1 = np.array(data1)
data1 = data1[:,0:3]
data2 = np.array(data2)
data2 = data2[:,0:3]
data3 = np.array(data3)
data3 = data3[:,0:3]

In [ ]:
A = ZscoreNormalization(data0)
cal_epsilon(A[0:120],A[120:],240,2)#验证组

In [ ]:
A = ZscoreNormalization(data1)
cal_epsilon(A[0:120],A[120:],240,2)#验证组

In [ ]:
A = ZscoreNormalization(data2)
cal_epsilon(A[0:120],A[120:],240,2)#验证组

In [ ]:
A = ZscoreNormalization(data3)
cal_epsilon(A[0:120],A[120:],480,2)#验证组